In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
import json
import random
excel_file = "./bop.xlsx"  
df = pd.read_excel(excel_file)

## 升序降序取topk 200

In [14]:
questions_and_queries = []
for i in range(5):
    flag = random.randint(0,1)
    a = random.randint(1, len(df) - 1)
    if(flag):
        question = f"What is the top {a} branches (along with their amounts) with the smallest customer_transaction_amount1."
        sql_query = f"""
        SELECT branch, customer_transaction_amount1
        FROM bop_test_cases
        ORDER BY customer_transaction_amount1 ASC
        LIMIT {a};
        """
    else:
        question = f"What is  the top {a} branches (along with their amounts) with the highest customer_transaction_amount1."
        sql_query = f"""
        SELECT branch, customer_transaction_amount1
        FROM bop_test_cases
        ORDER BY customer_transaction_amount1 DESC
        LIMIT {a};
        """
    questions_and_queries.append({"question": question, "sql_query": sql_query})
#     questions_and_queries.append({"db_id": "bop","question": question,"evidence": "", "SQL": ""})

# json_data = json.dumps(questions_and_queries,ensure_ascii=False,indent=2)

## 按条件筛选Return all data with branch as BJG and payment_type as '其他' 100

In [15]:
# questions_and_queries = []
column_mapping_reversed = {
    "Branch": "branch",
    "付款类型": "payment_type",
    "客户提供的交易编码1": "customer_transaction_code1",
    "客户提供的交易金额1": "customer_transaction_amount1",
    "客户提供的交易附言1": "customer_transaction_postscript1",
    "最终的交易编码1": "final_transaction_code1",
    "最终的相应金额1": "final_transaction_amount1",
    "最终的交易附言1": "final_transaction_postscript1",
}

for i in range(5):
    a = random.randint(0, len(df) - 1)
    random_column = random.choice(df.columns)
    column1 = random_column
    while column1 == random_column:
        random_column = random.choice(df.columns)
    column2 = random_column
    question = f"Return all data with {column_mapping_reversed[column1]} as {df[column1][a]} and {column_mapping_reversed[column2]} as {df[column2][a]}."
    sql_query = f"""
    SELECT *
        FROM bop_test_cases
        WHERE {column_mapping_reversed[column1]} = '{df[column1][a]}' AND {column_mapping_reversed[column2]} = '{df[column2][a]}';
    """
    questions_and_queries.append({"question": question, "sql_query": sql_query})
#     questions_and_queries.append({"question": question})
#     questions_and_queries.append({"db_id": "bop","question": question,"evidence": "", "SQL": ""})


In [87]:
with open('output.json', 'r') as f:
    data = json.load(f)
data[0]['question']

'What is the top 70 branches (along with their amounts) with the smallest customer_transaction_amount1.'

## 求和 Return the sum of all customer_transaction_amount1 with customer_transaction_code1 as ……     200

In [16]:
codes = list(set(df['客户提供的交易编码1']))
branches = list(set(df['Branch']))
payment_types = list(set(df['付款类型']))
# questions_and_queries = []
for i in range(5):
    random_code = random.choice(codes)
    random_branch = random.choice(branches)
    random_payment_type = random.choice(payment_types)

    choices = [
        (random_code, 'customer_transaction_code1'),
        (random_branch, 'branch'),
        (random_payment_type, 'payment_type')
    ]

    final_choice, condition = random.choice(choices)

    flag = random.randint(0,1)
    if (flag):
        question = f"Return the sum of all customer_transaction_amount1 with {condition} as {final_choice}."
        sql_query = f"""
            SELECT SUM(customer_transaction_amount1)
            FROM bop_test_cases
            WHERE {condition} = '{final_choice}';
        """
    else:
        question = f"Return the sum of all final_transaction_amount1 with {condition} as {final_choice}."
        sql_query = f"""
            SELECT SUM(final_transaction_amount1)
            FROM bop_test_cases
            WHERE {condition} = '{final_choice}';
        """
    questions_and_queries.append({"question": question, "sql_query": sql_query})
#     questions_and_queries.append({"db_id": "bop","question": question,"evidence": "", "SQL": ""})

# 将字典转换成 JSON 格式
# json_data = json.dumps(questions_and_queries,ensure_ascii=False)
# print(json_data)

## 减法 100 
todo 计算后的小数位数

In [10]:
codes = list(set(df['客户提供的交易编码1']))
branches = list(set(df['Branch']))
payment_types = list(set(df['付款类型']))
# questions_and_queries = []
for i in range(5):
    # 第一次随机选择
    random_code = random.choice(codes)
    random_branch = random.choice(branches)
    random_payment_type = random.choice(payment_types)

    choices = [
        (random_code, 'customer_transaction_code1'),
        (random_branch, 'branch'),
        (random_payment_type, 'payment_type')
    ]

    first_choice, first_condition = random.choice(choices)

    # 第二次随机选择
    random_code = random.choice(codes)
    random_branch = random.choice(branches)
    random_payment_type = random.choice(payment_types)

    choices = [
        (random_code, 'customer_transaction_code1'),
        (random_branch, 'branch'),
        (random_payment_type, 'payment_type')
    ]
    
    second_choice, second_condition = random.choice(choices)
    while second_choice == first_choice:
        second_choice, second_condition = random.choice(choices)
    if (flag):
        question = f"Subtract the sums of customer_transaction_amount1 for two sets of data: the first set where {first_condition} = {first_choice}, and the second set where {second_condition} = {second_choice},only the difference between these two sums is returned.Ensure the output matches the following regular expression pattern:^\d$."
        sql_query = f"""
            SELECT
                ABS(sum_first_condition - sum_second_condition) AS amount_difference
            FROM
                (
                    SELECT
                        SUM(customer_transaction_amount1) AS sum_first_condition
                    FROM
                        bop_test_cases
                    WHERE
                        {first_condition} = '{first_choice}'
                ) first_sum,
                (
                    SELECT
                        SUM(customer_transaction_amount1) AS sum_second_condition
                    FROM
                        bop_test_cases
                    WHERE
                        {second_condition} = '{second_choice}'
                ) second_sum;
        """
    else:
        question = f"Subtract the sums of final_transaction_amount1 for two sets of data: the first set where {first_condition} = {first_choice}, and the second set where {second_condition} = {second_choice}，only the difference between these two sums is returned.Ensure the output matches the following regular expression pattern:^\d$."
        sql_query = f"""
            SELECT
                ABS(sum_first_condition - sum_second_condition) AS amount_difference
            FROM
                (
                    SELECT
                        SUM(final_transaction_amount1) AS sum_first_condition
                    FROM
                        bop_test_cases
                    WHERE
                        {first_condition} = '{first_choice}'
                ) first_sum,
                (
                    SELECT
                        SUM(final_transaction_amount1) AS sum_second_condition
                    FROM
                        bop_test_cases
                    WHERE
                        {second_condition} = '{second_choice}'
                ) second_sum;
        """
    questions_and_queries.append({"question": question, "sql_query": sql_query})
#     questions_and_queries.append({"db_id": "bop","question": question,"evidence": "", "SQL": ""})


In [17]:
# questions_and_queries = []
for i in range(5):
    flag = random.randint(0,1)
    random_code = random.choice(codes)
    random_branch = random.choice(branches)
    random_payment_type = random.choice(payment_types)

    choices = [
        (random_code, 'customer_transaction_code1'),
        (random_branch, 'branch'),
        (random_payment_type, 'payment_type')
    ]

    final_choice, condition = random.choice(choices)
    if(flag):
        question = f"Return the smallest customer_transaction_amount1 of data with {condition} = {final_choice}."
        sql_query = f"""
        SELECT MIN(customer_transaction_amount1) AS smallest_amount
        FROM bop_test_cases
        WHERE {condition} = '{final_choice}';
        """
    else:
        question = f"Return the highest customer_transaction_amount1 of data with {condition} = {final_choice}."
        sql_query = f"""
        SELECT MAX(customer_transaction_amount1) AS highest_amount
        FROM bop_test_cases
        WHERE {condition} = '{final_choice}';
        """
    questions_and_queries.append({"question": question, "sql_query": sql_query})
#     questions_and_queries.append({"db_id": "bop","question": question,"evidence": "", "SQL": ""})


In [18]:
import json

# 写入到 JSON 文件，指定 ensure_ascii=False
with open('output.json', 'w', encoding='GBK') as f:
    json.dump(questions_and_queries, f, ensure_ascii=False,indent=2)


In [20]:
with open('output.json', 'r' ,encoding='GBK') as f:
    data = json.load(f)
questions = [data[i]['question'] for i in range(len(data))]
print(questions)

['What is the top 11 branches (along with their amounts) with the smallest customer_transaction_amount1.', 'What is the top 72 branches (along with their amounts) with the smallest customer_transaction_amount1.', 'What is the top 26 branches (along with their amounts) with the smallest customer_transaction_amount1.', 'What is the top 43 branches (along with their amounts) with the smallest customer_transaction_amount1.', 'What is  the top 83 branches (along with their amounts) with the highest customer_transaction_amount1.', 'Return all data with customer_transaction_amount1 as 77547.16 and final_transaction_amount1 as 77450.', 'Return all data with final_transaction_code1 as 228031-建筑、工程技术服务 and payment_type as 其他.', 'Return all data with branch as GZH and final_transaction_code1 as 121010-一般贸易.', 'Return all data with final_transaction_code1 as 121010-一般贸易 and customer_transaction_amount1 as 28662.0.', 'Return all data with customer_transaction_postscript1 as 一般贸易项下进口催化剂及化学制品货款支出 and